In [22]:
!pip install torchsummary

from torchsummary import summary
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop1 = nn.Dropout(0.1)
        self.drop2 = nn.Dropout(0.1)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.batch_norm2 = nn.BatchNorm2d(16)
        self.batch_norm3 = nn.BatchNorm2d(16)
        self.batch_norm4 = nn.BatchNorm2d(64)
        self.gap = nn.AdaptiveAvgPool2d(1)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3)

        self.fc = nn.Linear(in_features=1*1*64, out_features=10)

    def forward(self, x):
        x = self.conv1(x) # 28x28x1 => 24x24x32 | RF = 5
        x = F.relu(x)
        x = self.batch_norm1(x)

        x = self.conv2(x) # 24x24x32 => 22x22x16 | RF = 7
        x = F.relu(x)
        x = self.batch_norm2(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2) # 22x22x16 => 11x11x16 | RF = 14
        x = self.drop1(x)

        x = self.conv3(x) # 11x11x16 => 9x9x16 | RF = 16
        x = F.relu(x)
        x = self.batch_norm3(x)

        x = self.conv4(x) # 9x9x16 => 7x7x64 | RF = 18
        x = F.relu(x)
        x = self.batch_norm4(x)

        x = self.gap(x)
        x = self.drop2(x)

        x = x.reshape(-1, 64*1*1)

        x = self.fc(x) # 64*1*1 => 10

        return F.log_softmax(x, dim=1)

In [24]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 24, 24]             832
       BatchNorm2d-2           [-1, 32, 24, 24]              64
            Conv2d-3           [-1, 16, 22, 22]           4,624
       BatchNorm2d-4           [-1, 16, 22, 22]              32
           Dropout-5           [-1, 16, 11, 11]               0
            Conv2d-6             [-1, 16, 9, 9]           2,320
       BatchNorm2d-7             [-1, 16, 9, 9]              32
            Conv2d-8             [-1, 64, 7, 7]           9,280
       BatchNorm2d-9             [-1, 64, 7, 7]             128
AdaptiveAvgPool2d-10             [-1, 64, 1, 1]               0
          Dropout-11             [-1, 64, 1, 1]               0
           Linear-12                   [-1, 10]             650
Total params: 17,962
Trainable params: 17,962
Non-trainable params: 0
---------------------------------

In [25]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [26]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [27]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("Epoch: ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch:  1


loss=0.04502683877944946 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 41.44it/s]



Test set: Average loss: 0.0772, Accuracy: 9779/10000 (97.79%)

Epoch:  2


loss=0.09046034514904022 batch_id=937: 100%|██████████| 938/938 [00:19<00:00, 48.80it/s]



Test set: Average loss: 0.0548, Accuracy: 9827/10000 (98.27%)

Epoch:  3


...


Test set: Average loss: 0.0203, Accuracy: 9940/10000 (99.40%)

Epoch:  18


loss=0.011659398674964905 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 46.67it/s]



Test set: Average loss: 0.0184, Accuracy: 9936/10000 (99.36%)

Epoch:  19


loss=0.017939575016498566 batch_id=937: 100%|██████████| 938/938 [00:19<00:00, 48.72it/s]



Test set: Average loss: 0.0180, Accuracy: 9939/10000 (99.39%)

